# Decision Tree Classifier - From Scratch

## Problem Statement

You are given a very simple dataset that classifies drinks into three types: **Wine**, **Beer**, and **Whiskey**, based on their **alcohol content**, **sugar content**, and **color**. Your job is to build a **Decision Tree Classifier** that predicts the type of drink given its features.

## Load and Pre-process the Data

Alright, let's first load up the data into a NumPy matrix and split it into feature vectors and target labels, and also apply some ordinal encoding on the labels.

In [8]:
import numpy as np

data = [
    [12.0, 1.5, 1, 'Wine'],
    [5.0, 2.0, 0, 'Beer'],
    [40.0, 0.0, 1, 'Whiskey'],
    [13.5, 1.2, 1, 'Wine'],
    [4.5, 1.8, 0, 'Beer'],
    [38.0, 0.1, 1, 'Whiskey'],
    [11.5, 1.7, 1, 'Wine'],
    [5.5, 2.3, 0, 'Beer']
]

def preprocess_data(df):
    encoding = {'Wine':0,'Beer':1,'Whiskey':2}
    X = []
    y = []

    for row in df:
        X.append(row[:-1])
        y.append(encoding[row[3]])

    return np.array(X),np.array(y)

X, y = preprocess_data(data)
X

array([[12. ,  1.5,  1. ],
       [ 5. ,  2. ,  0. ],
       [40. ,  0. ,  1. ],
       [13.5,  1.2,  1. ],
       [ 4.5,  1.8,  0. ],
       [38. ,  0.1,  1. ],
       [11.5,  1.7,  1. ],
       [ 5.5,  2.3,  0. ]])

## Gini Impurity and Best Split

### Gini Impurity

We'll use **Gini Impurity** as the metric for splitting the dataset. The Gini Impurity is the probability of a new random point being misclassified according to the current distribution. Mathematically, the Gini Impurity of a dataset $D$ can is written as:

$Gini(D) = 1 - \sum_{i=1}^k p_i^2$

Where $k$ is the number of unique classes in $D$ and $p_i$ is the probability of a sample in $D$ belonging to class $i$.

If $D$ of size $n$ is split into $D_1$ and $D_2$ of sizes $n_1$ and $n_2$, the Gini Impurity can be defined as:

$Gini(D) = \frac{n_1}{n} Gini(D_1) + \frac{n_2}{n} Gini(D_2)$

### Best Split

To split the data, we will require a feature and the threshold value for that feature. In order to do this, we will do the following:

1. Iterate through all the feature vectors.

2. For each feature vector, iterate through all threshold values. The threshold values I am using will be the midpoint between two consecutive values in the sorted feature vector. So, for a feature vector $V = \{v_1,v_2,\dots ,v_m\}$, the thresholds will be $\frac{v_i + v_{i+1}}{2}$, where $i=1,2,\dots ,m$

3. Return the feature and threshold combination which gives the least Gini Impurity.

In [9]:
def gini_impurity(y):
    counts = {label:0 for label in set(y)}
    for y_i in y:
        counts[y_i] += 1
    n = len(y)

    impurity = 1
    for label,count in counts.items():
        impurity -= ((count/n)**2)

    return impurity

def return_best_split_gini(X,y):
    min_impurity = -1
    best_feature_index = None
    best_threshold = None

    m,n = X.shape
    for feature_index in range(n):
        feature_vector = X[:,feature_index]
        feature_vector = np.sort(feature_vector)
        for i in range(m-1):
            threshold = (feature_vector[i] + feature_vector[i+1])/2
            left_split = []
            right_split = []

            for j in range(m):
                if(X[j][feature_index] < threshold):
                    left_split.append(y[j])
                else:
                    right_split.append(y[j])

            impurity = len(left_split)/m * gini_impurity(left_split) + len(right_split)/m * gini_impurity(right_split)
            if min_impurity==-1 or (impurity < min_impurity):
                min_impurity = impurity
                best_feature_index = feature_index
                best_threshold = threshold

    return best_feature_index, best_threshold


## Implementing Recursive Tree Building

Defined below is the class `Node`, which has the following attributes:
- `feature_index` : Stores best feature index for splitting.
- `threshold` : Threshold value for splitting.
- `left` : Left child node.
- `right` : Right child node.
- `value` : Majority class label if leaf node.
- `min_samples_split` : Least number of samples required to split the Node further.

We also take a `max_depth` argument to limit the number of splits.

We directly initialise the object with $X$ and $y$ and it starts the splitting process by obtaining the best feauture and threshold, and then recursively splitting the child nodes until we reach leaf nodes.

Of course, a separate function can be made for fitting the data as is usually used, but this works too (I suppose).

Prediction is done by recursively taversing the tree until a leaf node is reached, where the value of the node is returned.

In [10]:
class Node:

    def __init__(self,X,y,max_depth=1):
        self.feature_index = None
        self.threshold = None
        self.left = None
        self.right = None
        self.value = None
        self.min_samples_split = 2

        m,_ = X.shape
        if len(np.unique(y))==1 or max_depth == 0 or m < self.min_samples_split:
            freqs = {label:0 for label in set(y)}
            max_freq = 0
            for i in range(m):
                freqs[y[i]] += 1

                if freqs[y[i]] > max_freq:
                    max_freq = freqs[y[i]]
                    self.value = y[i]

        else:
            self.feature_index, self.threshold = return_best_split_gini(X,y)

            left_X, left_y = [], []
            right_X, right_y = [], []
            m,_ = X.shape

            for i in range(m):
                if X[i][self.feature_index] < self.threshold:
                    left_X.append(X[i])
                    left_y.append(y[i])
                else:
                    right_X.append(X[i])
                    right_y.append(y[i])
            
            self.left = Node(np.array(left_X),np.array(left_y),max_depth-1)
            self.right = Node(np.array(right_X), np.array(right_y),max_depth-1)

    def predict(self,X):
        preds = []
        m,_ = X.shape
        if self.value == None:
            
            for i in range(m):
                if X[i][self.feature_index] < self.threshold:
                    preds.append(self.left.predict(X[i].reshape(1,-1))[0])
                else:
                    preds.append(self.right.predict(X[i].reshape(1,-1))[0])

        else:

            preds = [self.value] * m
        
        return np.array(preds)
    
    def print_tree(self, depth=0):
        indent = "  " * depth
        if self.value is not None:
            print(f"{indent}Return {self.value}")
        else:
            print(f"{indent}Feature[{self.feature_index}] < {self.threshold:.3f}?")
            if self.left:
                print(f"{indent}--> True:")
                self.left.print_tree(depth + 1)
            if self.right:
                print(f"{indent}--> False:")
                self.right.print_tree(depth + 1)

            

## Testing

Alright! Let's now test the data against a simple test dataset.

In [11]:
test_data = np.array([
    [6.0, 2.1, 0],   # Expected: Beer
    [39.0, 0.05, 1], # Expected: Whiskey
    [13.0, 1.3, 1]   # Expected: Wine
])

tree = Node(X,y,10)

def show_actual_pred(X):
    encoding = {0:'Wine',1:'Beer',2:'Whiskey'}
    preds = tree.predict(X)

    return np.array([encoding[pred] for pred in preds])

print(show_actual_pred(test_data))

['Beer' 'Whiskey' 'Wine']


Seems to work just fine!


# Bonus Tasks!

## Print the Tree in a Pretty Format

Alright, let's get to it!

In [12]:
# Already implemted the method in the Node class!
tree.print_tree()

Feature[0] < 8.500?
--> True:
  Return 1
--> False:
  Feature[0] < 25.750?
  --> True:
    Return 0
  --> False:
    Return 2


## Use Entropy instead of Gini as the Metric

Entropy for a dataset $D$ with $k$ classes and $p_i$ being the probability of a label belonging to class $i$ is defined as:

Entropy($D$) $= -\sum_{i=1}^k p_i log_2 (pi)$

So Entropy for a class split would be:

Entropy($D$) = $\frac{n_1}{n}$ Entropy($D_1$) + $\frac{n_2}{n}$ Entropy($D_2$)

In [13]:
def entropy(y):
    labels, counts = np.unique(y, return_counts=True)
    probs = counts / len(y)
    return -np.sum(probs * np.log2(probs))


def return_best_split_ent(X,y):
    min_impurity = -1
    best_feature_index = None
    best_threshold = None

    m,n = X.shape
    for feature_index in range(n):
        feature_vector = X[:,feature_index]
        feature_vector = np.sort(feature_vector)
        for i in range(m-1):
            threshold = (feature_vector[i] + feature_vector[i+1])/2
            left_split = []
            right_split = []

            for j in range(m):
                if(X[j][feature_index] < threshold):
                    left_split.append(y[j])
                else:
                    right_split.append(y[j])

            impurity = len(left_split)/m * entropy(left_split) + len(right_split)/m * entropy(right_split)
            if min_impurity==-1 or (impurity < min_impurity):
                min_impurity = impurity
                best_feature_index = feature_index
                best_threshold = threshold

    return best_feature_index, best_threshold


Now let's redefine our class and test it..

In [14]:
class Node:

    def __init__(self,X,y,max_depth=1):
        self.feature_index = None
        self.threshold = None
        self.left = None
        self.right = None
        self.value = None
        self.min_samples_split = 2

        m,_ = X.shape
        if len(np.unique(y))==1 or max_depth == 0 or m < self.min_samples_split:
            freqs = {label:0 for label in set(y)}
            max_freq = 0
            for i in range(m):
                freqs[y[i]] += 1

                if freqs[y[i]] > max_freq:
                    max_freq = freqs[y[i]]
                    self.value = y[i]

        else:
            self.feature_index, self.threshold = return_best_split_ent(X,y)

            left_X, left_y = [], []
            right_X, right_y = [], []
            m,_ = X.shape

            for i in range(m):
                if X[i][self.feature_index] < self.threshold:
                    left_X.append(X[i])
                    left_y.append(y[i])
                else:
                    right_X.append(X[i])
                    right_y.append(y[i])
            
            self.left = Node(np.array(left_X),np.array(left_y),max_depth-1)
            self.right = Node(np.array(right_X), np.array(right_y),max_depth-1)

    def predict(self,X):
        preds = []
        m,_ = X.shape
        if self.value == None:
            
            for i in range(m):
                if X[i][self.feature_index] < self.threshold:
                    preds.append(self.left.predict(X[i].reshape(1,-1))[0])
                else:
                    preds.append(self.right.predict(X[i].reshape(1,-1))[0])

        else:

            preds = [self.value] * m
        
        return np.array(preds)
    
    def print_tree(self, depth=0):
        indent = "  " * depth
        if self.value is not None:
            print(f"{indent}Return {self.value}")
        else:
            print(f"{indent}Feature[{self.feature_index}] < {self.threshold:.3f}?")
            if self.left:
                print(f"{indent}--> True:")
                self.left.print_tree(depth + 1)
            if self.right:
                print(f"{indent}--> False:")
                self.right.print_tree(depth + 1)

test_data = np.array([
    [6.0, 2.1, 0],   # Expected: Beer
    [39.0, 0.05, 1], # Expected: Whiskey
    [13.0, 1.3, 1]   # Expected: Wine
])

tree = Node(X,y,10)

def show_actual_pred(X):
    encoding = {0:'Wine',1:'Beer',2:'Whiskey'}
    preds = tree.predict(X)

    return np.array([encoding[pred] for pred in preds])

print(show_actual_pred(test_data))
            

['Beer' 'Whiskey' 'Wine']


Works!!

And I guess that concludes this task!